In [335]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from numpy import mean, std, absolute, arange

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [336]:
bike = pd.read_csv("Bike Sharing Dataset.csv", usecols = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
bike.info()
bike.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17379 non-null  object 
 1   yr          17379 non-null  int64  
 2   mnth        17379 non-null  object 
 3   hr          17379 non-null  int64  
 4   holiday     17379 non-null  object 
 5   weekday     17379 non-null  object 
 6   workingday  17379 non-null  object 
 7   weathersit  17379 non-null  object 
 8   temp        17379 non-null  float64
 9   atemp       17379 non-null  float64
 10  hum         17379 non-null  float64
 11  windspeed   17379 non-null  float64
 12  casual      17379 non-null  int64  
 13  registered  17379 non-null  int64  
 14  cnt         17379 non-null  int64  
dtypes: float64(4), int64(5), object(6)
memory usage: 2.0+ MB


,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,spring,0,january,0,no,saturday,no,clear,0.24,0.2879,0.81,0.0,3,13,16
1,spring,0,january,1,no,saturday,no,clear,0.22,0.2727,0.80,0.0,8,32,40
2,spring,0,january,2,no,saturday,no,clear,0.22,0.2727,0.80,0.0,5,27,32
3,spring,0,january,3,no,saturday,no,clear,0.24,0.2879,0.75,0.0,3,10,13
4,spring,0,january,4,no,saturday,no,clear,0.24,0.2879,0.75,0.0,0,1,1


In [337]:
bike.info()
print(bike.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17379 non-null  object 
 1   yr          17379 non-null  int64  
 2   mnth        17379 non-null  object 
 3   hr          17379 non-null  int64  
 4   holiday     17379 non-null  object 
 5   weekday     17379 non-null  object 
 6   workingday  17379 non-null  object 
 7   weathersit  17379 non-null  object 
 8   temp        17379 non-null  float64
 9   atemp       17379 non-null  float64
 10  hum         17379 non-null  float64
 11  windspeed   17379 non-null  float64
 12  casual      17379 non-null  int64  
 13  registered  17379 non-null  int64  
 14  cnt         17379 non-null  int64  
dtypes: float64(4), int64(5), object(6)
memory usage: 2.0+ MB
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp

In [338]:
bike = pd.get_dummies(bike, drop_first=True)
bike.head()

,yr,hr,temp,atemp,hum,windspeed,casual,registered,cnt,season_spring,...,weekday_monday,weekday_saturday,weekday_sunday,weekday_thursday,weekday_tuesday,weekday_wednesday,workingday_yes,weathersit_heavy rain,weathersit_light rain,weathersit_misty
0,0,0,0.24,0.2879,0.81,0.0,3,13,16,True,...,False,True,False,False,False,False,False,False,False,False
1,0,1,0.22,0.2727,0.80,0.0,8,32,40,True,...,False,True,False,False,False,False,False,False,False,False
2,0,2,0.22,0.2727,0.80,0.0,5,27,32,True,...,False,True,False,False,False,False,False,False,False,False
3,0,3,0.24,0.2879,0.75,0.0,3,10,13,True,...,False,True,False,False,False,False,False,False,False,False
4,0,4,0.24,0.2879,0.75,0.0,0,1,1,True,...,False,True,False,False,False,False,False,False,False,False


In [340]:
x = bike.drop(['cnt','casual','registered'], axis=1)
y = bike['cnt']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [341]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [342]:
cv = RepeatedKFold(n_splits=10, n_repeats=10)
model = RidgeCV(alphas=range(1,100), cv=cv, scoring='neg_mean_absolute_error')

model.fit(x_train, y_train)

print(model.alpha_)
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

99
0.4127513740380334
0.3911391613821624


In [343]:
cv = RepeatedKFold(n_splits=10, n_repeats=10)
model = LassoCV(alphas=range(1, 100), cv=cv)

model.fit(x_train, y_train)

print(model.alpha_)
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

1
0.4113342334275103
0.3906604609712958
